# Part of Speech Tagging with Recurrent Neural Networks
ubc ml club 3/1/2018

### Below: Determiner - Noun - Verb - Determiner - Noun
![part of speech tagging](https://i.stack.imgur.com/6pdIT.png)

In [ ]:
import csv

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from nltk.tokenize import word_tokenize

## Part of Speech Tags

Each tag denotes a certain part of speech, which are described in English terms in the tag_descriptions.

In [ ]:
# List of all part-of-speech tags.
tags = [
  'LS',
  'TO',
  'VBN',
  "''",
  'WP',
  'UH',
  'VBG',
  'JJ',
  'VBZ',
  '--',
  'VBP',
  'NN',
  'DT',
  'PRP',
  ':',
  'WP$',
  'NNPS',
  'PRP$',
  'WDT',
  '(',
  ')',
  '.',
  ',',
  '``',
  '$',
  '#',
  'RB',
  'RBR',
  'RBS',
  'VBD',
  'IN',
  'FW',
  'RP',
  'JJR',
  'JJS',
  'PDT',
  'MD',
  'VB',
  'WRB',
  'NNP',
  'EX',
  'NNS',
  'SYM',
  'CC',
  'CD',
  'POS',
]

# Descriptions for each part-of-speech tag.
tag_descriptions = [
  'list item marker',
  '"to" as preposition or infinitive marker',
  'verb, past participle',
  'closing quotation mark',
  'WH-pronoun',
  'interjection',
  'verb, present participle or gerund',
  'adjective or numeral, ordinal',
  'verb, present tense, 3rd person singular',
  'dash',
  'verb, present tense, not 3rd person singular',
  'noun, common, singular or mass',
  'determiner',
  'pronoun, personal',
  'colon or ellipsis',
  'WH-pronoun, possessive',
  'noun, proper, plural',
  'pronoun, possessive',
  'WH-determiner',
  'opening parenthesis',
  'closing parenthesis',
  'sentence terminator',
  'comma',
  'opening quotation mark',
  'dollar',
  'octothorp',
  'adverb',
  'adverb, comparative',
  'adverb, superlative',
  'verb, past tense',
  'preposition or conjunction, subordinating',
  'foreign word',
  'particle',
  'adjective, comparative',
  'adjective, superlative',
  'pre-determiner',
  'modal auxiliary',
  'verb, base form',
  'Wh-adverb',
  'noun, proper, singular',
  'existential there',
  'noun, common, plural',
  'symbol',
  'conjunction, coordinating',
  'numeral, cardinal',
  'genitive marker',
]

In [ ]:
# Build a map of tag to its numerical index.
tags_to_idxs = {tags[i]: i for i in range(len(tags))}

# Build a map of tag to its description.
tags_to_descriptions = {tags[i]: tag_descriptions[i] for i in range(len(tags))}

## Loading Data

We have 3 sets of data:
- **Training Data**: list of pairs of (Sentence, Part of Speech Tags), used to train the model.
- **Validation Data**: list of pairs of (Sentence, Part of Speech Tags), hidden from the model during training, we use this to check that we aren't overfitting to training data.
- **Test Data**: list of Sentence, we make predictions on this to submit to the contest.

In [ ]:
def load_data(data_path='data/train_data.csv', labels_path='data/train_labels.csv'):
    '''
    Load data as a pairs, [(sentence, labels), ...].
    '''
    data = []
    
    f_data = open(data_path, 'r')
    sents = f_data.readlines()
    
    for sent in sents:
        tokens = word_tokenize(sent)
        data.append(tokens)

    # If labels_path is not provided, just return the data alone (e.g. for test data).
    if labels_path is None:
        return data

    f_labels = open(labels_path, 'r')
    
    labels_reader = csv.reader(f_labels, delimiter=' ')
    labels = list(labels_reader)
    
    return list(zip(data, labels))

In [ ]:
train_data = load_data()
val_data = load_data(data_path='data/val_data.csv', labels_path='data/val_labels.csv')
test_data = load_data(data_path='data/test_data.csv', labels_path=None)

# Take a look at what a training data pair looks like.
train_data[0]

## Building Vocabulary

We want to assign a unique index (numerical representation) for each word that is present in our data to pass into our model, which operates with numbers, not text.

Here we simply give every unique case-insensitive word we see an index for simplicity.

However this is not ideal, for example stemmed words such as 'run' and 'running' would be treated as separate unique words despite being related, and typos would also be treated as unique words.

In [ ]:
def build_vocab(train_data, val_data, test_data):
    '''
    Builds a lexicographically sorted vocabulary using unique words from the datasets.
    '''
    words = set()
    words_to_idxs = {}
    
    all_sents = [sent for (sent, tags) in train_data] + [sent for (sent, tags) in val_data] + test_data
    
    for sent in all_sents:
        for word in sent:
            word = word.upper()
            if word not in words:
                words.add(word)

    words = sorted(list(words))

    for i, word in enumerate(words):
        words_to_idxs[word] = i

    return words, words_to_idxs

In [ ]:
words, words_to_idxs = build_vocab(train_data, val_data, test_data)

### Prepare Sequence Function

This helper function turns a sequence (either a list of words, or a list of POS tags) into lists of numerical indices, and converts it to a PyTorch Variable to be used by our model.

In [ ]:
def prepare_sequence(seq, to_idxs):
    '''
    Turn a sequence of tokens into their respective indices, and create a PyTorch tensor Variable.
    '''
    idxs = [to_idxs[w.upper()] for w in seq]
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)

## Part-of-Speech Tagger Model

We define an LSTM (long short term memory) part-of-speech model.

An LSTM is a type of recurrent neural network, which for each word in a sentence:
- Remembers some information about the words seen up to this point (memory/hidden state).
    - This is useful because each word's part of speech depends on the previous words.
    - e.g. 'over the bridge': bridge is an prepositional object because we observed a preposition directly before it.
    - The hidden state represents all knowledge learned from words of the sentence seen by the model up to this point, and is used to make the prediction for the current word.
- Generates an output using the memory. This will be the prediction.

### Overview

![lstm](https://i.imgur.com/9yfjOxY.png)


### Internals

![lstm internals](https://cdn-images-1.medium.com/max/1600/1*J5W8FrASMi93Z81NlAui4w.png)

In [ ]:
class POSTagger(nn.Module):
    '''
    Our LSTM part of speech tagger model.
    '''
    def __init__(self, embedding_size, hidden_size, vocab_size, tagset_size):
        super(POSTagger, self).__init__()
        self.hidden_size = hidden_size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_size)
        
        self.lstm = nn.LSTM(embedding_size, hidden_size)
        self.classifier = nn.Linear(hidden_size, tagset_size)
        
    def forward(self, sentence):
        embeddings = self.word_embeddings(sentence)
        output, self.hidden = self.lstm(
            embeddings.view(len(sentence), 1, -1),
            self.hidden,
        )
        output = self.classifier(output.view(len(sentence), -1))
        output = F.log_softmax(output, dim=1)
        return output
    
    def init_hidden(self):
        return (
            Variable(torch.zeros(1, 1, self.hidden_size)),
            Variable(torch.zeros(1, 1, self.hidden_size)),
        )

### Evaluate Function

Given a model and a dataset, computes the average loss across all sentences.

### NLL Loss
The loss being used is called Negative Log Likelihood, which is a number that is correlated with accuracy, but has better numerical properties for training than simply using the accuracy percentage.

In [ ]:
def evaluate(model, data, words_to_idxs, tags_to_idxs):
    '''
    Compute the average loss across the given dataset.
    '''
    loss_sum = 0.0
    criterion = nn.NLLLoss()

    for sentence, tags in data:
        x = prepare_sequence(sentence, words_to_idxs)
        y = prepare_sequence(tags, tags_to_idxs)

        output = model(x)
        loss = criterion(output, y)
        loss_sum += loss.data[0]

    return loss_sum / len(data)

### Train Function

Given:
- model (our LSTM tagger)
- criterion (an object that represents the loss function, with backpropagation built in)
- optimizer (an object containing an algorithm that takes adjusts the model to improve it after each training sentence seen)
- our data
- number of epochs (how many times we should review the training data, analogous a human re-reading a textbook chapter for better understanding).

Our train function trains the model, and prints how the training and validation loss changes every so often.

In [ ]:
def train(
    model,
    criterion,
    optimizer,
    train_data,
    val_data,
    words_to_idxs,
    tags_to_idxs,
    epochs=5,
    print_every=1000
):
    # An epoch is one entire pass through the entire training data set.
    for epoch in range(epochs):
        for i, (sentence, tags) in enumerate(train_data):
            # Reset the gradients and hidden state for each new sentence.
            optimizer.zero_grad()
            model.hidden = model.init_hidden()
            
            # Convert both the sentence and labels to input Tensors.
            x = prepare_sequence(sentence, words_to_idxs)
            y = prepare_sequence(tags, tags_to_idxs)

            # Make prediction.
            y_pred = model(x)
            
            # Compute the loss (how good our prediction is).
            loss = criterion(y_pred, y)
            # Backprop to slightly adjust our model in hopes of improving predictions.
            loss.backward()
            optimizer.step()
            
            # Print intermediate results every so often.
            if (i+1) % print_every == 0:
                model.eval()
                val_loss = evaluate(model, val_data, words_to_idxs, tags_to_idxs)
                model.train()
                print ('Epoch [%d/%d], Iter [%d/%d], Train Loss: %.4f, Validation Loss: %.4f' 
                        %(epoch+1, epochs, i+1, len(train_data), loss.data[0], val_loss))

### Train our model

In [ ]:
# Dimensions of our embedding vectors and hidden state vectors.
EMBEDDING_SIZE = 32
HIDDEN_SIZE = 32

# Create our model.
model = POSTagger(EMBEDDING_SIZE, HIDDEN_SIZE, len(words_to_idxs), len(tags_to_idxs))
# Set model to train mode (not important with this specific model, but still good practice).
model.train()

# Create our loss function (NLL or negative log likelihood loss).
criterion = nn.NLLLoss()

# Learning rate affects how big of a 'step' each adjustment to the model makes.
# - Too large of a learning rate will cause instability during training (e.g. jumpy loss).
# - Too small of a learning rate will make loss improve slowly and possibly get stuck
#   in a local optimum and no longer improve.
#
# Using a more sophisticated optimizer may also improve your results, e.g.
# - SGD with momentum
# - Adam
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Train our model.
# Note: increasing epochs will improve results up to a certain point,
# but will take more time to train. You can play around 
train(model, criterion, optimizer, train_data, val_data, words_to_idxs, tags_to_idxs, epochs=1)

## Prediction

Using our trained model, we make part of speech predictions for every sentence in our test data, and save it to predictions.csv

In [ ]:
import numpy as np

def predict(model, sentences, words_to_idxs):
    predictions = []

    for sent in sentences:
        x = prepare_sequence(sent, words_to_idxs)
        y_pred = model(x)
        y_pred = y_pred.data.numpy()
        
        # Choose the tag index with highest log probability for each word.
        y_pred = np.argmax(y_pred, axis=1)

        y_pred = [tags[i] for i in y_pred]
        predictions.append(y_pred)

    return predictions


def save_predictions(predictions, path='predictions.csv'):
    with open('predictions.csv', 'w+') as f:
        for pred in predictions:
            pred = ' '.join(pred)
            f.write(pred + '\n')

In [ ]:
predictions = predict(model, test_data, words_to_idxs)
save_predictions(predictions)